In [1]:
import json
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import yaml
from box import ConfigBox
from sklearn.metrics import f1_score
from tqdm import tqdm


### Set web serice URL

In [2]:
# local testing
API_URL = "http://0.0.0.0:8080" 
# testing deployed service
# API_URL = "https://<YOUR_APP_NAME>.fly.dev" 

API_URL

'http://0.0.0.0:8080'

### Load feature columns and data

In [3]:
def load_params(params_path):
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
        params = ConfigBox(params)
    return params


proj_path = Path(os.getcwd()).parent.absolute()
params = load_params(proj_path/'params.yaml')
feat_cols = params.base.feat_cols
targ_col = params.base.targ_col
feat_cols, targ_col

(BoxList(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']),
 'Exited')

In [4]:
df = pd.concat([pd.read_csv(proj_path/'data'/'raw'/f'Churn_Modelling_{country}.csv') for country in ['France', 'Spain']])
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
2,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
3,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
4,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0


### Test `/predict` endpoint by sending one sample request

In [5]:
my_obj = {
    "data": [
            {
      "CreditScore": 619,
      "Age": 42,
      "Tenure": 2,
      "Balance": 0,
      "NumOfProducts": 1,
      "HasCrCard": 1,
      "IsActiveMember": 1,
      "EstimatedSalary": 101348.88
            }
        ]
    }
x = requests.post(API_URL + '/predict', json = my_obj)
if x.ok:
    probs = x.json()
else:
    x.raise_for_status()

probs

[0.6985236842105264]

### Create a functions for calling `/predict` endpoint

In [6]:
def get_prob(obj):
    x = requests.post(API_URL + '/predict', json = obj)
    if x.ok:
        probs = x.json()
    else:
        x.raise_for_status()
    return probs

get_prob(my_obj)

[0.6985236842105264]

### Check what would f1-scores look like if we send the data from the same geographies that the model was trained on 

In [7]:
import time


def send_sample_requests(df):
    f1_score_list = []
    for _ in tqdm(range(50)):
        df_sample = df.sample(n=60)
        y_true = df_sample[targ_col].values
        obj = {"data": df_sample[feat_cols].to_dict('records')}
        probs = get_prob(obj)
        y_pred = np.array([prob < 0.5 for prob in probs]).astype(int)
        f1 = f1_score(y_true, y_pred)
        f1_score_list.append(f1)
        time.sleep(1)
    return f1_score_list
        
f1_score_list = send_sample_requests(df)
f1_score_list

100%|██████████| 50/50 [00:51<00:00,  1.02s/it]


[0.8695652173913044,
 0.9333333333333333,
 0.888888888888889,
 0.9090909090909091,
 0.8333333333333333,
 1.0,
 0.8695652173913043,
 0.9333333333333333,
 0.9600000000000001,
 0.9523809523809523,
 0.923076923076923,
 0.9333333333333333,
 0.8,
 0.9090909090909091,
 0.8421052631578948,
 0.962962962962963,
 0.9565217391304348,
 1.0,
 0.9600000000000001,
 0.9523809523809523,
 0.7272727272727273,
 1.0,
 0.9473684210526316,
 0.923076923076923,
 0.8333333333333334,
 0.9166666666666666,
 0.9565217391304348,
 0.9655172413793104,
 0.888888888888889,
 0.88,
 0.8571428571428571,
 0.7272727272727273,
 0.8571428571428571,
 1.0,
 0.7142857142857143,
 0.9523809523809523,
 1.0,
 1.0,
 0.7777777777777778,
 0.8571428571428571,
 0.923076923076923,
 0.8695652173913044,
 0.9166666666666666,
 0.8888888888888888,
 0.9600000000000001,
 0.7692307692307693,
 0.7272727272727273,
 0.9473684210526316,
 0.9333333333333333,
 0.72]

### What would f1-scores look like for input data from new geography?

In [8]:
df_germany =  pd.read_csv(proj_path/'data'/'more_data'/'Churn_Modelling_Germany.csv')
time.sleep(20) # this is to create a pause between requests. Later it'll be easier to distinguish the two types if requests
f1_score_list = send_sample_requests(df_germany)
f1_score_list


100%|██████████| 50/50 [00:51<00:00,  1.02s/it]


[0.47619047619047616,
 0.4166666666666667,
 0.5925925925925926,
 0.44444444444444436,
 0.5384615384615384,
 0.5517241379310346,
 0.4827586206896552,
 0.6956521739130435,
 0.20689655172413793,
 0.6428571428571428,
 0.5263157894736842,
 0.37037037037037035,
 0.4666666666666666,
 0.48,
 0.4848484848484849,
 0.4999999999999999,
 0.6666666666666666,
 0.3225806451612903,
 0.42857142857142855,
 0.4666666666666667,
 0.5,
 0.5384615384615385,
 0.2608695652173913,
 0.5714285714285715,
 0.2222222222222222,
 0.48484848484848486,
 0.42857142857142855,
 0.47368421052631576,
 0.27586206896551724,
 0.5517241379310345,
 0.6666666666666667,
 0.3846153846153846,
 0.6666666666666666,
 0.6,
 0.4,
 0.47619047619047616,
 0.625,
 0.4666666666666667,
 0.22222222222222224,
 0.5882352941176471,
 0.6666666666666666,
 0.5777777777777778,
 0.5185185185185185,
 0.3157894736842105,
 0.6249999999999999,
 0.44444444444444436,
 0.21052631578947367,
 0.625,
 0.5,
 0.47058823529411764]

### ... as expected, the values are much lower.
### But, typically, we can't compute model metrics on production data right away 
### because ground truth labels might not be available until much-much later


### All we can do is try and look at "proxy" metrics that measure statistical differences (distances) between train data and production data

In [9]:
x = requests.get(API_URL + '/drift_data')
if x.ok:
    data = x.json()
else:
    x.raise_for_status()

df_p_vals = pd.DataFrame(json.loads(data))
df_p_vals

,time,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,2024-09-23 18:57:38.717679,0.067774,0.751316,0.998375,7.679846e-01,0.998382,0.870738,1.000000,0.520943
1,2024-09-23 18:57:39.734591,0.517316,0.267510,0.954832,1.717239e-01,0.925950,0.998416,1.000000,0.944133
2,2024-09-23 18:57:40.769570,0.787392,0.987857,0.999548,4.668937e-01,0.999997,1.000000,0.819814,0.855582
3,2024-09-23 18:57:41.794473,0.992581,0.937580,0.735384,8.298411e-01,1.000000,1.000000,0.712223,0.626845
4,2024-09-23 18:57:42.817834,0.980853,0.549738,0.945256,3.547196e-01,0.570102,0.993182,0.497856,0.776069
...,...,...,...,...,...,...,...,...,...
95,2024-09-23 19:00:04.809746,0.938457,0.157454,0.589638,0.000000e+00,0.925950,1.000000,1.000000,0.008658
96,2024-09-23 19:00:05.837170,0.934194,0.873823,0.691418,0.000000e+00,1.000000,0.387848,1.000000,0.920333
97,2024-09-23 19:00:06.861032,0.495984,0.998168,0.801245,0.000000e+00,1.000000,0.998416,0.901523,0.300678
98,2024-09-23 19:00:07.880275,0.931539,0.361617,0.735384,0.000000e+00,1.000000,0.999993,1.000000,0.782443


### Values below the threshold (e.g. 0.05) indicate data drift

In [10]:
import plotly_express as px

fig = px.line(df_p_vals, x='time', y=feat_cols)
fig.add_hline(y=0.05, line_color='red')